<a href="https://colab.research.google.com/github/PavlovGeorgiy/portfolio/blob/main/%D0%B4%D0%B8%D0%BF%D0%BB%D0%BE%D0%BC%D0%BD%D1%8B%D0%B9_%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82_%D0%BA%D0%BE%D0%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

▶ **Необходимо загрузить файлы [по ссылке](https://drive.google.com/drive/folders/12rtXpuselFVLiR0HWy4ENgq0rTvIB0az?usp=sharing)** ◀

In [ ]:
import pandas as pd
import numpy as np

Алгоритм решения бизнес-задачи:
1. Создание общего датафрейма:
  * чтение файлов
  * удаление лишних столбцов
  * переименование столбцов для конкатинации
  * очистка данных от лишних символов
  * исправление типа данных
2. Исправление данных в столбце "заказчик"
3. Заполнение пропусков
4. Выручка, прибыль, маржинальность и количество запросов
5. Кол-во сделок по форматам и % от кол-ва успешных сделок
6. Кол-во сделок по типу
7. Юнит-экономика
8. Общие расходы
9. Точка безубыточности
10. Топ заказчиков
11. Рейтинг форматов
12. анализ отказов
13. Выгрузка итоговых данных для составления дашбордов

# Создание датафрейма

Данные представлены csv файлами. За 2017 год данные есть с июня месяца. А датасет 2022-2023 года идет одним csv файлом. Поэтому обработаем их отдельно

In [ ]:
dff17 = pd.DataFrame()

for i in [2017]:
  for j in ['june','july','aug', 'sept', 'oct', 'nov', 'dec']:
    df17 = pd.read_csv('fin_data_' + str(i) + j + '.csv')
    dff17 = pd.concat([dff17,df17])
dff17 = dff17.drop(['Unnamed: 6', 'администратор', 'Unnamed: 10', 'Unnamed: 12',
            'Unnamed: 14', 'Unnamed: 16', 'Unnamed: 18', 'Чист.Приб',
            'Unnamed: 20', 'Unnamed: 3', 'Unnamed: 5', 'Unnamed: 7',
            'Unnamed: 9', 'Unnamed: 11', 'Unnamed: 13', 'Unnamed: 15',
            'Unnamed: 17', 'Unnamed: 19', 'Unnamed: 21', 'Unnamed: 23', 'ЧП',
            'Unnamed: 25', 'Unnamed: 4', 'Unnamed: 8', 'Unnamed: 22',
            'Unnamed: 24', 'Unnamed: 26'], axis=1)
dff17 = dff17.rename(columns = {
    'дата': 'дата события',
    'мероприятие': 'тип',
    'заказ': 'выручка',
    'откат': 'агентские',
    'призы': 'типография',
    'прочее': 'доп расх'})

In [ ]:
# очистим датафрейм от лишних символов
rep = [' ', '-', '₽', ',', ' ₽', '\xa0']
col = ['выручка', 'персонал', 'транспорт', 'агентские',
       'типография', 'налог', 'доп расх', 'прибыль', 'амортизация', 'маркетинг']
for i in rep:
  for j in col:
    dff17[col] = dff17[col].replace(to_replace = rep, value = '', regex = True)

In [ ]:
# заменим пустые, нулевые и текстовые значения  на NaN

lst_rep = ['руб', ' ', '0.00', '', '0', '0.0', '  ']
lst_col = ['дата события', 'тип', 'выручка', 'персонал', 'транспорт', 'агентские',
       'типография', 'налог', 'доп расх', 'прибыль', 'амортизация',
       'маркетинг', 'заказчик']

for i in lst_col:
  for j in lst_rep:
    dff17.loc[dff17[i] == j, i] = np.NaN

dff17 = dff17.dropna(how='all')

In [ ]:
df22 = pd.read_csv('fin_data_2022jan.csv')
df22 = df22[df22['дата запроса'].notnull()]
df22 = df22.drop(['маржа, %', 'разндат'], axis=1)
df22 = df22.rename(columns = {
    'прочее': 'доп развлечения',
    'маржа, руб.': 'прибыль',
    'оборотка': 'выручка'})

In [ ]:
rep = ['₽', '\xa0', ' ']
col = ['игры', 'ФК', 'доп развлечения', 'выручка', 'персонал',
       'транспорт', 'агентские', 'налог', 'доп расх', 'типография', 'прибыль', 'гостей']
for i in rep:
  for j in col:
    df22[col] = df22[col].replace(to_replace = rep, value ='', regex = True)

In [ ]:
dff = pd.DataFrame()

for i in [2018, 2019, 2020, 2021]:
  for j in ['jan',
            'feb',
            'march',
            'apr',
            'may',
            'june',
            'july',
            'aug',
            'sept',
            'oct',
            'nov',
            'dec']:
    df = pd.read_csv('fin_data_' + str(i) + j + '.csv')
    dff = pd.concat([dff, df])
dff = dff.drop(['маржа, %', 'расходы', 'Unnamed: 16', 'Unnamed: 17',
                'Unnamed: 18', 'Unnamed: 14', 'Unnamed: 19', 'аниматоры',
                'п/о', 'кто работает'], axis = 1)
dff = dff.rename(columns = {
    'дата': 'дата события',
    'маржа, руб.': 'прибыль',
    'оборотка': 'выручка',
    'откат': 'агентские',
    'призы': 'типография',
    'прочее': 'доп расх',
    'другое': 'доп развлечения'})

In [ ]:
rep = ['₽', '\xa0', ' ']
col = ['выручка', 'персонал', 'транспорт',
       'агентские', 'типография', 'доп расх', 'налог', 'амортизация',
       'маркетинг', 'прибыль', 'игры', 'ФК', 'карты XXL', 'АХ',
       'комиссия банка', 'фанты', 'доп развлечения']
for i in rep:
  for j in col:
    dff[col] = dff[col].replace(to_replace = rep, value = '', regex = True)

In [ ]:
dff = pd.concat([dff17, dff, df22])
lst_rep = [' ', '0.00', '', '0', '0.0', '  ']
lst_col = ['выручка', 'персонал', 'транспорт',
       'агентские', 'типография', 'доп расх', 'налог', 'амортизация',
       'маркетинг', 'прибыль', 'игры', 'ФК', 'карты XXL', 'АХ',
       'комиссия банка', 'фанты', 'доп развлечения']

for i in lst_col:
  for j in lst_rep:
    dff.loc[dff[i] == j, i] = np.NaN

dff = dff.replace(to_replace = ',', value = '.', regex = True)
dff = dff.dropna(how='all')
dff.reset_index(inplace=True, drop=True)

In [ ]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 698 entries, 0 to 697
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   дата события             697 non-null    object 
 1   тип                      157 non-null    object 
 2   выручка                  358 non-null    object 
 3   персонал                 588 non-null    object 
 4   транспорт                603 non-null    object 
 5   агентские                327 non-null    object 
 6   типография               71 non-null     object 
 7   налог                    298 non-null    object 
 8   доп расх                 202 non-null    object 
 9   прибыль                  683 non-null    object 
 10  амортизация              247 non-null    object 
 11  маркетинг                247 non-null    object 
 12  заказчик                 652 non-null    object 
 13  игры                     407 non-null    object 
 14  ФК                       1

In [ ]:
# исправим типы данных на float
col1 = ['выручка', 'персонал', 'транспорт', 'агентские',
       'типография', 'налог', 'доп расх', 'прибыль', 'амортизация',
       'маркетинг', 'игры', 'ФК', 'карты XXL', 'АХ', 'комиссия банка', 'фанты',
       'доп развлечения', 'гостей', 'количество аттракционов',
       'кол-во персонала', 'продолжительность']
for cols in col1:
  dff[col1] = dff[col1].astype('float')

# исправим столбцы с датами и временем
dff['дата события'] = pd.to_datetime(dff['дата события'], errors='coerce')
dff['дата запроса'] = pd.to_datetime(dff['дата запроса'])
dff['время начала'] = pd.to_datetime(dff['время начала'], format="%H:%M")
dff['время окончания'] = pd.to_datetime(dff['время окончания'], format="%H:%M")

<ipython-input-11-53adc041673a>:11: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  dff['дата события'] = pd.to_datetime(dff['дата события'], errors='coerce')
<ipython-input-11-53adc041673a>:12: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  dff['дата запроса'] = pd.to_datetime(dff['дата запроса'])


In [ ]:
 dff['прочие расходы'] = dff[['доп расх', 'комиссия банка']].sum(axis = 1)
 dff['распечатки'] = dff[['типография', 'фанты']].sum(axis = 1)
 dff['допники'] = dff[['доп развлечения', 'карты XXL', 'АХ']].sum(axis = 1)

In [ ]:
dff = dff.drop(['фанты', 'комиссия банка', 'карты XXL',
                'АХ', 'типография','доп расх', 'доп развлечения'], axis=1)

In [ ]:
rep1 = ['ИА', 'ИСФ', 'Иа']
rep2 = ['игры + ФК', 'Комбо', 'ИА + ФК']
rep3 = ['заказ игры', 'мороженое', 'ТМ', 'SMM']

for i in rep1, rep2, rep3:
    dff['тип'] = dff['тип'].replace(to_replace = rep1, value = 'игры', regex = True)
    dff['тип'] = dff['тип'].replace(to_replace = rep2, value = 'игры и ФК', regex = True)
    dff['тип'] = dff['тип'].replace(to_replace = rep3, value = 'допники', regex = True)
dff.loc[dff['тип'] == 'игры + ФК', 'тип'] = 'игры и ФК'

# Исправим значения в столбце "заказчик"

In [ ]:
dff['заказчик'] = dff['заказчик'].replace(to_replace='\d', value = '', regex = True)
dff['заказчик'] = dff['заказчик'].str.title().str.strip()

In [ ]:
dff.loc[dff['заказчик'] == 'Bla Bla Bar', 'заказчик'] = 'Андрей Бла Бла Бар'
dff.loc[dff['заказчик'] == 'Galadance', 'заказчик'] = 'Galladance'
dff.loc[(dff['заказчик'] == 'Идея Фикс') |\
        (dff['заказчик'] == 'Идея Фикс Бтл'), 'заказчик'] = 'Ideafix'
dff.loc[(dff['заказчик'] == 'Zotov') |\
        (dff['заказчик'] == 'Zotov-Event'), 'заказчик'] = 'Zotov Event'
dff.loc[(dff['заказчик'] == 'Абожина') |\
        (dff['заказчик'] == 'Юля Абожина'), 'заказчик'] = 'Юлия Абожина'
dff.loc[(dff['заказчик'] == 'Адажио (Ксения)') |\
        (dff['заказчик'] == 'Адажио Ксения'), 'заказчик'] = 'Ксения Лосева'
dff.loc[(dff['заказчик'] == 'Настя Захарова') |\
        (dff['заказчик'] == 'Захарова'), 'заказчик'] = 'Анастасия Захарова'
dff.loc[(dff['заказчик'] == 'Аня Баранова') |\
        (dff['заказчик'] == 'Баранова Аня'), 'заказчик'] = 'Анна Баранова'
dff.loc[(dff['заказчик'] == 'Аня Смычагина') |\
        (dff['заказчик'] == 'Смычагина'), 'заказчик'] = 'Анна Смычагина'
dff.loc[(dff['заказчик'] == 'Арина (Бонифаций)') |\
        (dff['заказчик'] == 'Бонифаций'), 'заказчик'] = 'Арина Зырянова'
dff.loc[(dff['заказчик'] == 'Башня Свободы') |\
        (dff['заказчик'] == 'Ип Букреев') |\
        (dff['заказчик'] == 'Легион') |\
        (dff['заказчик'] == 'Ск Легион') |\
        (dff['заказчик'] == 'Тютюгина') |\
        (dff['заказчик'] == 'Жанна Тютюгина'), 'заказчик'] = 'Голос.девелопмент'
dff.loc[(dff['заказчик'] == 'Боришпольская') |\
        (dff['заказчик'] == 'Женя Боришпольская'), 'заказчик'] = 'Евгения Боришпольская'
dff.loc[(dff['заказчик'] == 'Сева Ведущий') |\
        (dff['заказчик'] == 'Сева Прохоренков'), 'заказчик'] = 'Всеволод Прохоренков'
dff.loc[(dff['заказчик'] == 'Гепарт') |\
        (dff['заказчик'] == 'Оксана Гепарт'), 'заказчик'] = 'Оксана Руденко'
dff.loc[(dff['заказчик'] == 'Джи Трак') |\
        (dff['заказчик'] == 'Дтв') |\
        (dff['заказчик'] == 'Ооо Дтв') |\
        (dff['заказчик'] == 'Светалана Макарова'), 'заказчик'] = 'Светлана Макарова (ООО ДТВ)'
dff.loc[dff['заказчик'] == 'Елена Бабенко', 'заказчик'] = 'Чкз'
dff.loc[(dff['заказчик'] == 'Елена Галеева') |\
        (dff['заказчик'] == 'Империя Праздника') |\
        (dff['заказчик'] == 'Юля Ахметзянова') |\
        (dff['заказчик'] == 'Юля Кононенко'), 'заказчик'] = 'Юлия Ахметзянова'
dff.loc[(dff['заказчик'] == 'Лена Фаенок') |\
        (dff['заказчик'] == 'Фаенок'), 'заказчик'] = 'Елена Фаенок'
dff.loc[(dff['заказчик'] == 'Юля Зонтович') |\
        (dff['заказчик'] == 'Зонтович'), 'заказчик'] = 'Юлия Зонтович'
dff.loc[(dff['заказчик'] == 'Инглина') |\
        (dff['заказчик'] == 'Наташа Инглина'), 'заказчик'] = 'Наталья Инглина'
dff.loc[(dff['заказчик'] == 'Ира Косарева') |\
        (dff['заказчик'] == 'Косарева') |\
        (dff['заказчик'] == 'Косарева Ира'), 'заказчик'] = 'Ирина Косарева'
dff.loc[(dff['заказчик'] == 'Катя Куракова') |\
        (dff['заказчик'] == 'Куракова'), 'заказчик'] = 'Екатерина Куракова'
dff.loc[(dff['заказчик'] == 'Кубышкин') |\
        (dff['заказчик'] == 'Саша Кубышкин'), 'заказчик'] = 'Александр Кубышкин'
dff.loc[(dff['заказчик'] == 'Мелиот') |\
        (dff['заказчик'] == 'Мелиот (Артюх)'), 'заказчик'] = 'Оксана Артюх'
dff.loc[(dff['заказчик'] == 'Юлия Ворон') |\
        (dff['заказчик'] == 'Юлия Ворона'), 'заказчик'] = 'Юлия Воронова'
dff.loc[dff['заказчик'] == 'Куклина Анастасия', 'заказчик'] = 'Анастасия Куклина'
dff.loc[dff['заказчик'] == 'Анна Ромкор', 'заказчик'] = 'Анна Бородина'
dff.loc[dff['заказчик'] == 'Антонина', 'заказчик'] = 'Антонина Черкасова'
dff.loc[dff['заказчик'] == 'Аня Панова', 'заказчик'] = 'Анна Панова'
dff.loc[dff['заказчик'] == 'Баулькин', 'заказчик'] = 'Константин Баулькин'
dff.loc[dff['заказчик'] == 'Валя Париев', 'заказчик'] = 'Валентин Париев'
dff.loc[dff['заказчик'] == 'Вова Скоков', 'заказчик'] = 'Владимир Скоков'
dff.loc[dff['заказчик'] == 'Квитко', 'заказчик'] = 'Владлена Квитко'
dff.loc[dff['заказчик'] == 'Гафарова Альбина Наилевна', 'заказчик'] = 'Ит Энигма'
dff.loc[dff['заказчик'] == 'Самохвалов', 'заказчик'] = 'Денис Самохвалов'
dff.loc[dff['заказчик'] == 'Дима Березка', 'заказчик'] = 'Дима Зайков'
dff.loc[dff['заказчик'] == 'Катя Васильева', 'заказчик'] = 'Екатерина Васильева'
dff.loc[dff['заказчик'] == 'Кружево', 'заказчик'] = 'Лена Павлова'
dff.loc[dff['заказчик'] == 'Савушкина', 'заказчик'] = 'Елена Савушкина'
dff.loc[dff['заказчик'] == 'Мюллер', 'заказчик'] = 'Инна Мюллер'
dff.loc[dff['заказчик'] == 'Ирина Фролова (Выбери Радио)', 'заказчик'] = 'Ирина Фролова'
dff.loc[dff['заказчик'] == 'Исаева', 'заказчик'] = 'Лена Исаева'
dff.loc[dff['заказчик'] == 'Козловский', 'заказчик'] = 'Эдгар Козловский'
dff.loc[dff['заказчик'] == 'Контстантин', 'заказчик'] = 'Константин'
dff.loc[dff['заказчик'] == 'Ксения Мелиот', 'заказчик'] = 'Ксения Утивалиева'
dff.loc[dff['заказчик'] == 'Ксюша Чистякова', 'заказчик'] = 'Ксения Чистякова'
dff.loc[dff['заказчик'] == 'Шакирова Лена', 'заказчик'] = 'Лена Шакирова'
dff.loc[dff['заказчик'] == 'Мазитов', 'заказчик'] = 'Руслан Мазитов'
dff.loc[dff['заказчик'] == 'Меренков', 'заказчик'] = 'Макс Меренков'
dff.loc[dff['заказчик'] == 'Репникова', 'заказчик'] = 'Маша Репникова'
dff.loc[dff['заказчик'] == 'Смолин', 'заказчик'] = 'Миша Смолин'
dff.loc[dff['заказчик'] == 'Эминова Ольга', 'заказчик'] = 'Ольга Эминова'
dff.loc[dff['заказчик'] == 'Савченко', 'заказчик'] = 'Паша Савченко'
dff.loc[dff['заказчик'] == 'Рустам Габрахманов', 'заказчик'] = 'Рустам Габдрахманов'
dff.loc[dff['заказчик'] == 'Юургу', 'заказчик'] = 'Светлана Владимировна'
dff.loc[dff['заказчик'] == 'Саша Морозов', 'заказчик'] = 'Александр Морозов'
dff.loc[dff['заказчик'] == 'Оранжевый Слон', 'заказчик'] = 'Кирилл Феоктистов'
dff.loc[dff['заказчик'] == 'Юрова Анна (Невеста)', 'заказчик'] = 'Анна Юрова'

# Заполним пропуски по следующему алгоритму:
1. Все значения 0.0 заменим на NaN
2. Все пустые значения в "стадия" заменим на "мероприятие завершено".
3. Заполнить столбец "тип" имеющимися категориями по условиям:
* если заполнен только столбец "игры" - присваивать категорию "игры"
* если заполнен только столбец "ФК" - присваивать категорию "ФК"
* если если заполнены столбцы "игры" и "ФК" - присваивать категорию "игры и ФК"
* если если заполнены столбцы "игры" и "допники" - присваивать категорию "игры и допники"
* если заполнен столбец "допники" - присваивать категорию "допники"
* если заполнены все столбцы - категория "полный"
* в остальных случаях тип - "неопределен"

4. Заполнение пропусков по выручке:
Если выручка содержит NaN - записывать значением сумму значений стобцов "игры", "ФК" и "допники"
5. Заполнение Nan в столбцах "игры", "ФК" и "допники":
Если выручка не NaN, а в столбцах  - NaN, то в зависимости от значения в столбце "тип":
- дублировать значение выручки в соответствующий столбец
- для категорий "игры и ФК", "Фк и допники", "игры и допники",  - делить значение выручки на 2 и разносить по соответствующим столбцам
- для категории "полный" - делить выручку на 3 и разносить по всем столбцам



In [ ]:
col = ['выручка', 'персонал', 'транспорт', 'агентские',
       'налог', 'прибыль', 'амортизация', 'маркетинг', 'игры',
       'ФК', 'гостей', 'количество аттракционов',
       'кол-во персонала', 'продолжительность', 'прочие расходы',
       'распечатки', 'допники']

for i in col:
  dff.loc[dff[i] == 0.0, i] = np.NaN

In [ ]:
dff.loc[dff['стадия'].isna(), 'стадия'] = 'мероприятие завершено'

## Заполняем столбец "тип"

In [ ]:
game = (dff['тип'].isna()) & (dff['игры'].notna()) & (dff['ФК'].isna()) & (dff['допники'].isna())
fk = (dff['тип'].isna()) & (dff['игры'].isna()) & (dff['ФК'].notna()) & (dff['допники'].isna())
games_fk = (dff['тип'].isna()) & (dff['игры'].notna()) & (dff['ФК'].notna()) & (dff['допники'].isna())
game_additional = (dff['тип'].isna()) & (dff['игры'].notna()) & (dff['ФК'].isna()) & (dff['допники'].notna())
fk_additional = (dff['тип'].isna()) & (dff['игры'].isna()) & (dff['ФК'].notna()) & (dff['допники'].notna())
additional = (dff['тип'].isna()) & (dff['игры'].isna()) & (dff['ФК'].isna()) & (dff['допники'].notna())
full = (dff['тип'].isna()) & (dff['игры'].notna()) & (dff['ФК'].notna()) & (dff['допники'].notna())

In [ ]:
dff.loc[game, 'тип'] = 'игры'
dff.loc[fk, 'тип'] = 'ФК'
dff.loc[games_fk, 'тип'] = 'игры и ФК'
dff.loc[game_additional, 'тип'] = 'игры и допники'
dff.loc[fk_additional, 'тип'] = 'ФК и допники'
dff.loc[additional, 'тип'] = 'допники'
dff.loc[full, 'тип'] = 'полный'
dff.loc[dff['тип'].isna(), 'тип'] = 'неопределен'

In [ ]:
col2 = ['тип', 'заказчик', 'формат', 'стадия', 'источник', 'площадка']

for cols in col2:
  dff[col2] = dff[col2].astype('category')

## Заполняем столбец "выручка"

In [ ]:
dff.loc[dff['выручка'] == 0.0, 'выручка'] = np.NaN

In [ ]:
dff.loc[dff['выручка'].isna(), 'выручка'] = dff[['игры', 'ФК', 'допники']].sum(axis = 1)

## Заполнение Nan в столбцах "игры", "ФК" и "допники"

In [ ]:
def data_change(dff):

  if dff['тип'] == 'полный':
    dff['игры'] = dff['выручка'] / 3
    dff['ФК'] = dff['выручка'] / 3
    dff['допники'] = dff['выручка'] / 3
  elif dff['тип'] == 'ФК и допники':
    dff['ФК'] = dff['выручка'] / 2
    dff['допники'] = dff['выручка'] / 2
  elif dff['тип'] == 'игры и ФК':
    dff['игры'] = dff['выручка'] / 2
    dff['ФК'] = dff['выручка'] / 2
  elif dff['тип'] == 'игры и допники':
    dff['игры'] = dff['выручка'] / 2
    dff['допники'] = dff['выручка'] / 2
  elif dff['тип'] == 'допники':
    dff['допники'] = dff['выручка']
  elif dff['тип'] == 'ФК':
    dff['ФК'] = dff['выручка']
  elif dff['тип'] == 'игры':
    dff['игры'] = dff['выручка']

  return dff

dff = dff.apply(data_change, axis=1)
dff[['выручка', 'игры', 'ФК', 'допники', 'прибыль']]

,выручка,игры,ФК,допники,прибыль
0,25000.0,NaN,25000.0,NaN,18650.0
1,27000.0,27000.0,NaN,NaN,20800.0
2,7000.0,7000.0,NaN,NaN,5500.0
3,16000.0,16000.0,NaN,NaN,9900.0
4,12000.0,12000.0,NaN,NaN,8600.0
...,...,...,...,...,...
693,20000.0,20000.0,NaN,NaN,12580.0
694,12500.0,6250.0,NaN,6250.0,8900.0
695,35500.0,35500.0,NaN,NaN,26070.0
696,1500.0,NaN,NaN,1500.0,1140.0


In [ ]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 698 entries, 0 to 697
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   дата события             696 non-null    datetime64[ns]
 1   тип                      698 non-null    object        
 2   выручка                  698 non-null    float64       
 3   персонал                 584 non-null    float64       
 4   транспорт                601 non-null    float64       
 5   агентские                317 non-null    float64       
 6   налог                    283 non-null    float64       
 7   прибыль                  683 non-null    float64       
 8   амортизация              240 non-null    float64       
 9   маркетинг                240 non-null    float64       
 10  заказчик                 652 non-null    object        
 11  игры                     532 non-null    float64       
 12  ФК                       185 non-nul

# Перечень показателей для расчета:
* выручка и прибыль за всё время
* средняя выручка и прибыль за год, месяц и за одно мероприятие
* маржинальность общая (прибыль/выручка * 100)
* общее количество запросов, кол-во выигранных сделок
* выручка, прибыль и маржинальность по направлениям деятельности
* кол-во заказов по направлениям и % от общего числа
* сумму расходов по категориям (unit-экономика): общая и по направлениям, в суммах и в %
* точка безубыточности
* топ 10 заказчиков по выручке и прибыли
* популярные форматы (количество и сумма по выручке и прибыли)
* отказы: количество, сумма выручки и конверсия от общего количества запросов

Для расчета показателей оставим только сделки со стадией "мероприятие завершено"

In [ ]:
metrics = dff[dff['стадия'] == 'мероприятие завершено']
metrics

,дата события,тип,выручка,персонал,транспорт,агентские,налог,прибыль,амортизация,маркетинг,...,время окончания,источник,гостей,количество аттракционов,кол-во персонала,продолжительность,площадка,прочие расходы,распечатки,допники
0,2017-06-01,ФК,25000.0,2000.0,500.0,2500.0,1350.0,18650.0,3730.0,1865.0,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-06-01,игры,27000.0,3600.0,500.0,NaN,NaN,20800.0,4160.0,2080.0,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,600.0,1500.0,NaN
2,2017-06-01,игры,7000.0,1000.0,500.0,NaN,NaN,5500.0,1100.0,550.0,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-06-01,игры,16000.0,NaN,3100.0,3000.0,NaN,9900.0,1980.0,990.0,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-06-01,игры,12000.0,1200.0,1000.0,1200.0,NaN,8600.0,1720.0,860.0,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,2023-11-03,ФК,58000.0,12000.0,1500.0,NaN,2320.0,39680.0,NaN,NaN,...,1900-01-01 20:00:00,телефон,20.0,3.0,3.0,4.0,Клубничная 3,NaN,2500.0,NaN
676,2023-03-24,игры,10000.0,2100.0,500.0,NaN,600.0,6800.0,NaN,NaN,...,1900-01-01 15:00:00,VK моя страница,250.0,4.0,3.0,1.0,с. Долгодеревенское ул Свердловская 15. Районн...,NaN,NaN,NaN
677,2023-02-04,игры,12500.0,3600.0,500.0,NaN,500.0,7900.0,NaN,NaN,...,1900-01-01 13:00:00,whatsapp,150.0,5.0,3.0,2.0,Уральская молния,NaN,NaN,NaN
678,2023-05-04,ФК,60000.0,12500.0,1339.0,NaN,3600.0,42561.0,NaN,NaN,...,1900-01-01 23:00:00,телефон,60.0,3.0,4.0,3.0,Видгоф,NaN,NaN,NaN


# Выручка, прибыль, маржинальность и количество запросов

In [ ]:
# суммы по всем годам

metrics[['выручка', 'игры', 'ФК', 'допники','прибыль']].sum()

выручка    8144290.00
игры       4778021.00
ФК         2921719.00
допники     364850.00
прибыль    4712063.39
dtype: float64

In [ ]:
# средние показатели за год
mean_year_metrics = metrics.groupby(metrics['дата события'].dt.year )[['дата события','выручка', 'игры', 'ФК', 'допники', 'прибыль']]
round(mean_year_metrics.sum().mean(),2)

<ipython-input-28-1509f277e766>:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  round(mean_year_metrics.sum().mean(),2)


выручка    1158470.00
игры        681503.00
ФК          417388.43
допники      52121.43
прибыль     672883.63
dtype: float64

In [ ]:
# средние показатели за месяц

month_revenue = metrics.groupby([pd.Grouper(key='дата события', freq='M')]).agg(month_revenue=('выручка', 'sum')).reset_index()
month_games = metrics.groupby([pd.Grouper(key='дата события', freq='M')]).agg(month_games=('игры', 'sum')).reset_index()
month_fk = metrics.groupby([pd.Grouper(key='дата события', freq='M')]).agg(month_fk=('ФК', 'sum')).reset_index()
month_dop = metrics.groupby([pd.Grouper(key='дата события', freq='M')]).agg(month_dop=('допники', 'sum')).reset_index()
month_cash = metrics.groupby([pd.Grouper(key='дата события', freq='M')]).agg(month_cash=('прибыль', 'sum')).reset_index()

print(f'Средняя выручка в месяц: {round(float(month_revenue.mean()))}\n'
f'Средняя выручка по играм в месяц: {round(float(month_games.mean()))}\n'
f'Средняя выручка по ФК в месяц: {round(float(month_fk.mean()))}\n'
f'Средняя выручка по допникам в месяц: {round(float(month_dop.mean()))}\n'
f'Средняя прибыль в месяц: {round(float(month_cash.mean()))}')

Средняя выручка в месяц: 103965
Средняя выручка по играм в месяц: 61161
Средняя выручка по ФК в месяц: 37458
Средняя выручка по допникам в месяц: 4678
Средняя прибыль в месяц: 60387


<ipython-input-29-c869df50c79d>:9: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  print(f'Средняя выручка в месяц: {round(float(month_revenue.mean()))}\n'
<ipython-input-29-c869df50c79d>:10: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  f'Средняя выручка по играм в месяц: {round(float(month_games.mean()))}\n'
<ipython-input-29-c869df50c79d>:11: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  f'Средняя выручка по ФК в месяц: {round(float(month_fk.mean()))}\n'
<ipython-input-29-c869df50c79d>:12: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  f'Средняя выручка по допникам в месяц: {round(float(month_

In [ ]:
# средние чеки и средняя прибыль с одного мероприятия
metrics[['выручка', 'игры', 'ФК', 'допники','прибыль']].mean()

выручка    17073.983229
игры       12913.570270
ФК         23005.661417
допники    13512.962963
прибыль     9941.062004
dtype: float64

In [ ]:
# маржинальность общая

round(metrics['прибыль'].sum() / metrics['выручка'].sum() * 100,2)

57.86

In [ ]:
all = len(dff) # всего запросов
sucsess_len = len(dff.loc[dff['стадия'] == 'мероприятие завершено']) # успешных сделок
convers = round(len(dff.loc[dff['стадия'] == 'мероприятие завершено']) / len(dff) * 100,2) # конверсия сделок, %
print(f'Всего запросов: {all} \n'
f'Из них успешных сделок: {sucsess_len} \n'
f'Конверсия в успех: {convers} %')

Всего запросов: 698 
Из них успешных сделок: 477 
Конверсия в успех: 68.34 %


# Кол-во сделок по форматам и % от кол-ва успешных сделок

In [ ]:
metrics.groupby(['формат']).agg({'выручка': 'count'}).\
assign(f_all=lambda x: round(x['выручка'] / sucsess_len *100,2)).\
sort_values(by='выручка', ascending=False).reset_index()

,формат,выручка,f_all
0,корпоратив,88,18.45
1,день рождения,27,5.66
2,клиентский день,23,4.82
3,свадьба,21,4.40
4,конференция,18,3.77
5,день рождения детский,16,3.35
6,выпускной,6,1.26
7,другое,6,1.26
8,детский праздник,5,1.05
9,презентация,5,1.05


# Кол-во сделок по типу

In [ ]:
metrics.groupby(['тип']).agg({'выручка': 'count'}).\
sort_values(by='выручка', ascending = False).reset_index().head(10)

,тип,выручка
0,игры,321
1,ФК,88
2,игры и ФК,33
3,игры и допники,12
4,допники,9
5,неопределен,8
6,полный,4
7,ФК и допники,2


# Unit-экономика

In [ ]:
unit = metrics[['формат', 'выручка','персонал', 'транспорт', 'агентские',
       'налог', 'амортизация', 'маркетинг', 'прочие расходы',
       'распечатки', 'прибыль']]
unit

,формат,выручка,персонал,транспорт,агентские,налог,амортизация,маркетинг,прочие расходы,распечатки,прибыль
0,NaN,25000.0,2000.0,500.0,2500.0,1350.0,3730.0,1865.0,NaN,NaN,18650.0
1,NaN,27000.0,3600.0,500.0,NaN,NaN,4160.0,2080.0,600.0,1500.0,20800.0
2,NaN,7000.0,1000.0,500.0,NaN,NaN,1100.0,550.0,NaN,NaN,5500.0
3,NaN,16000.0,NaN,3100.0,3000.0,NaN,1980.0,990.0,NaN,NaN,9900.0
4,NaN,12000.0,1200.0,1000.0,1200.0,NaN,1720.0,860.0,NaN,NaN,8600.0
...,...,...,...,...,...,...,...,...,...,...,...
672,день рождения,58000.0,12000.0,1500.0,NaN,2320.0,NaN,NaN,NaN,2500.0,39680.0
676,корпоратив,10000.0,2100.0,500.0,NaN,600.0,NaN,NaN,NaN,NaN,6800.0
677,благотворительность,12500.0,3600.0,500.0,NaN,500.0,NaN,NaN,NaN,NaN,7900.0
678,конференция,60000.0,12500.0,1339.0,NaN,3600.0,NaN,NaN,NaN,NaN,42561.0


In [ ]:
# общая unit-экономика

unit_in_percent = pd.DataFrame()
col = ['персонал', 'транспорт', 'агентские', 'налог', 'амортизация',
       'маркетинг', 'прочие расходы', 'распечатки']

for i in col:
  unit_in_percent = round(unit[col].sum() / unit['выручка'].sum() * 100,2)
print(f'Затраты (в %) на:\n'
f'{unit_in_percent[col]}')

Затраты (в %) на:
персонал          21.14
транспорт          5.84
агентские          5.43
налог              2.14
амортизация        4.93
маркетинг          2.46
прочие расходы     2.18
распечатки         0.92
dtype: float64


In [ ]:
# Юнит-экономика в суммах
unit.sum()

<ipython-input-36-71f226025f4f>:2: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  unit.sum()


выручка           8144290.00
персонал          1721750.00
транспорт          475909.00
агентские          441902.40
налог              173960.43
амортизация        401402.70
маркетинг          200701.35
прочие расходы     177432.03
распечатки          74665.00
прибыль           4712063.39
dtype: float64

In [ ]:
# unit-экономика по направлениям
unit_types = unit.groupby(['формат']).sum().reset_index()
col = ['персонал', 'транспорт', 'агентские', 'налог', 'амортизация',
       'маркетинг', 'прочие расходы', 'распечатки', 'прибыль']
for c in col:
  unit_types[c] = round(unit_types[c] / unit_types['выручка'] * 100,2)
unit_types.sort_values(by='выручка',ascending = False)

,формат,выручка,персонал,транспорт,агентские,налог,амортизация,маркетинг,прочие расходы,распечатки,прибыль
9,корпоратив,2008513.0,23.58,6.72,5.64,3.23,0.0,0.0,1.10,1.14,58.59
3,день рождения,600500.0,21.97,5.09,5.58,2.03,0.0,0.0,0.32,1.67,63.34
7,клиентский день,410700.0,24.42,5.39,2.67,2.42,0.0,0.0,0.50,1.12,63.48
8,конференция,335200.0,19.57,3.47,2.51,4.45,0.0,0.0,0.31,0.75,68.94
11,свадьба,231300.0,28.73,5.27,6.33,1.31,0.0,0.0,1.29,1.93,55.14
4,день рождения детский,224400.0,21.08,10.03,5.44,2.55,0.0,0.0,0.90,1.34,58.67
10,презентация,117600.0,23.30,2.72,3.49,4.98,0.0,0.0,1.02,0.00,64.49
5,детский праздник,101200.0,17.89,8.89,12.77,3.01,0.0,0.0,2.47,0.00,54.97
12,тимбилдинг,96500.0,22.18,7.25,9.33,4.10,0.0,0.0,1.04,2.69,53.41
13,юбилей,89500.0,29.94,4.21,16.76,0.00,0.0,0.0,3.02,2.64,43.42


# Общие расходы

In [ ]:
expenses = pd.DataFrame()

for i in [2017, 2018, 2019, 2020, 2021, 2022]:
  exp = pd.read_csv('fin_data_' + str(i) + 'expenses' + '.csv')
  expenses = expenses.rename(columns = {'месяц': 'дата'})
  expenses = pd.concat([expenses, exp])

expenses.reset_index(inplace=True, drop=True)
expenses

,дата,наименование,категория,сумма
0,6/2017,прокси,маркетинг,270
1,6/2017,сошлхамер,маркетинг,1440
2,6/2017,капча,маркетинг,595
3,6/2017,симки,маркетинг,900
4,6/2017,столики,амортизация,6700
...,...,...,...,...
176,2022-02-01,хз,прочее,300 ₽
177,2022-03-01,маркетинг,маркетинг,17 050 ₽
178,2022-06-22,стретч,амортизация,690 ₽
179,2022-07-01,типография,маркетинг,1 700 ₽


In [ ]:
rep = ['₽', '\xa0', ' ']
col = ['категория',	'сумма']
for i in rep:
  for j in col:
    expenses[col] = expenses[col].replace(to_replace = rep, value = '', regex = True)
expenses['сумма'] = expenses['сумма'].replace(to_replace = ',', value = '.', regex = True)

In [ ]:
expenses.loc[expenses['категория'] == 'налоги', 'категория'] = 'налог'
expenses.loc[expenses['категория'] == 'прочие', 'категория'] = 'прочее'
expenses.dropna()

,дата,наименование,категория,сумма
0,6/2017,прокси,маркетинг,270
1,6/2017,сошлхамер,маркетинг,1440
2,6/2017,капча,маркетинг,595
3,6/2017,симки,маркетинг,900
4,6/2017,столики,амортизация,6700
...,...,...,...,...
176,2022-02-01,хз,прочее,300
177,2022-03-01,маркетинг,маркетинг,17050
178,2022-06-22,стретч,амортизация,690
179,2022-07-01,типография,маркетинг,1700


In [ ]:
expenses['дата'] = pd.to_datetime(expenses['дата'], errors='coerce')
expenses['категория'] = expenses['категория'].astype('category')
expenses['сумма'] = expenses['сумма'].astype(float)

In [ ]:
expenses.groupby(['категория']).sum().sort_values(by='сумма', ascending = False).reset_index()

<ipython-input-42-97ab011e441f>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  expenses.groupby(['категория']).sum().sort_values(by='сумма', ascending = False).reset_index()


,категория,сумма
0,амортизация,463465.0
1,маркетинг,213572.5
2,налог,90496.0
3,прочее,26617.4
4,бухгалтерия,13100.0
5,операционные,4376.0


# точка безубыточности

Изначально в проект было вложено 220 000 рублей. Найдем дату, когда прибыль от деятельности стала нулевой с учетом инвестиционного вложения

In [ ]:
import datetime
from datetime import date
from datetime import datetime
from datetime import timedelta

total = 0
for row in metrics[['дата события', 'прибыль']].itertuples():
  total += row[2]
  if total >= 220000:
    print('Инвестиции в размере 220 тысяч рублей окупились', row[1])
    break
tdelta = (row[1]-metrics['дата события'][0]).days / 30
print(f'Срок окупаемости: {tdelta} месяца')

Инвестиции в размере 220 тысяч рублей окупились 2017-08-06 00:00:00
Срок окупаемости: 2.2 месяца


# Топ заказчиков по выручке, прибыли и количеству заказов

In [ ]:
metrics.groupby(['заказчик']).agg({'выручка': 'sum', 'прибыль': 'sum'}).\
sort_values(by='выручка', ascending = False).reset_index().head(10)

,заказчик,выручка,прибыль
0,Инна Мюллер,441000.0,220323.00
1,Кирилл Феоктистов,330000.0,217580.00
2,Юлия Ахметзянова,277363.0,142588.00
3,Анастасия Захарова,222300.0,125942.00
4,Ирина Косарева,173400.0,98064.00
5,Zotov Event,154800.0,75693.00
6,Светлана Макарова (ООО ДТВ),153000.0,81971.00
7,Голос.девелопмент,143500.0,75439.00
8,Евгения Боришпольская,120300.0,70141.00
9,Таврия,110600.0,78316.25


In [ ]:
metrics.groupby(['заказчик']).agg({'выручка': 'count'}).\
sort_values(by='выручка', ascending = False).reset_index().head(10)

,заказчик,выручка
0,Инна Мюллер,14
1,Ирина Косарева,13
2,Zotov Event,10
3,Голос.девелопмент,9
4,Таврия,8
5,Юлия Ахметзянова,8
6,Елена Фаенок,8
7,Софья Окольнишникова,7
8,Чкз,7
9,Анастасия Захарова,7


# отказы: количество, сумма выручки и конверсия от общего количества запросов

In [ ]:
unsucess = dff.loc[(dff['стадия'] == 'причина не выяснена') |\
        (dff['стадия'] == 'заказали другое') |\
        (dff['стадия'] == 'по бюджету не прошли') |\
        (dff['стадия'] == 'отменилось мероприятие') |\
        (dff['стадия'] == 'отказались от организатора')]

unsucess

,дата события,тип,выручка,персонал,транспорт,агентские,налог,прибыль,амортизация,маркетинг,...,время окончания,источник,гостей,количество аттракционов,кол-во персонала,продолжительность,площадка,прочие расходы,распечатки,допники
252,2020-01-01,игры,12000.0,2000.0,300.0,NaN,NaN,9580.0,NaN,NaN,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,120.0,NaN,NaN
254,2020-04-01,игры,12000.0,2000.0,300.0,NaN,NaN,9580.0,NaN,NaN,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,120.0,NaN,NaN
256,2020-07-01,игры,12000.0,2000.0,300.0,NaN,NaN,9580.0,NaN,NaN,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,120.0,NaN,NaN
258,2020-01-17,игры,8500.0,2000.0,500.0,850.0,NaN,5150.0,NaN,NaN,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
261,2020-01-27,игры,7000.0,1400.0,500.0,700.0,NaN,4400.0,NaN,NaN,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670,2023-04-03,игры,25000.0,6600.0,2500.0,2500.0,900.0,12500.0,NaN,NaN,...,NaT,VK моя страница,NaN,3.0,3.0,4.0,Солнечная долина,NaN,NaN,NaN
673,2023-03-14,ФК,22500.0,7000.0,2500.0,7500.0,600.0,4900.0,NaN,NaN,...,NaT,телефон,20.0,1.0,2.0,3.0,Увильды,NaN,NaN,NaN
674,2023-03-18,игры,15000.0,NaN,NaN,NaN,NaN,15000.0,NaN,NaN,...,NaT,whatsapp,20.0,3.0,2.0,2.0,гостиница алмаз,NaN,NaN,NaN
675,2023-03-20,игры,7500.0,NaN,NaN,NaN,NaN,7500.0,NaN,NaN,...,NaT,VK моя страница,120.0,3.0,3.0,1.0,Малахит,NaN,NaN,NaN


In [ ]:
print(f"В стадиях неуспеха {len(unsucess)} сделок на сумму {unsucess['выручка'].sum()} рублей.\n"
f"{(unsucess['выручка'].sum() / dff['выручка'].sum() * 100).round(2)}% сделок были неудачными от общего числа запросов")

В стадиях неуспеха 180 сделок на сумму 4035850.0 рублей.
31.12% сделок были неудачными от общего числа запросов


In [ ]:
# сумма выручки по неуспешным статусам
unsucess.groupby(['стадия'])['выручка'].sum().sort_values(ascending = False)

стадия
причина не выяснена           1196900.0
заказали другое               1047000.0
по бюджету не прошли           879800.0
отменилось мероприятие         650100.0
отказались от организатора     262050.0
Name: выручка, dtype: float64

In [ ]:
# выручка в % от общей суммы выручки неуспешных сделок по статусам неуспеха
(unsucess.groupby(['стадия'])['выручка'].sum().sort_values(ascending = False) /\
 unsucess['выручка'].sum() * 100).round(0)

стадия
причина не выяснена           30.0
заказали другое               26.0
по бюджету не прошли          22.0
отменилось мероприятие        16.0
отказались от организатора     6.0
Name: выручка, dtype: float64

In [ ]:
# кол-во неуспешных сделок по статусам
unsucess.groupby(['стадия'])['выручка'].count().sort_values(ascending = False)

стадия
причина не выяснена           55
заказали другое               48
по бюджету не прошли          38
отменилось мероприятие        33
отказались от организатора     6
Name: выручка, dtype: int64

# Выгружаем датафреймы для дальнейшей работы

In [ ]:
dff['маржинальность'] = (dff['прибыль'] / dff['выручка']*100).round(2)
dff = dff.reindex(
    columns = dff.columns.tolist() + ['команда', 'реквизит', 'описание', 'первичное'])

In [ ]:
# создадим файл с юнит-экономикой
unit_economy = dff[['выручка', 'игры', 'ФК', 'допники', 'персонал', 'транспорт',
                    'агентские', 'налог', 'распечатки', 'прочие расходы',
                    'амортизация', 'маркетинг', 'прибыль', 'маржинальность']]
unit_economy.to_csv('unit_economy.csv')

In [ ]:
# создадим файл с характеристиками мероприятий
event_characteristics = dff[['дата события', 'время начала', 'время окончания',
                             'продолжительность', 'формат', 'гостей',
                             'количество аттракционов', 'реквизит','кол-во персонала',
                             'команда', 'площадка', 'описание']]
event_characteristics.to_csv('event_characteristics.csv')

In [ ]:
# файл с общими характеристиками сделок
deal_characteristics = dff[['дата запроса', 'стадия', 'тип', 'заказчик', 'источник', 'первичное']]
deal_characteristics.to_csv('deal_characteristics.csv')

In [ ]:
# создадим файл с транзакциями
expenses = expenses.reindex(columns = expenses.columns.tolist() + ['тип'])
expenses.to_csv('transactions.csv', index=False)